# 応用情報技術者試験 ベクトル化ノートブック

このノートブックは Google Colab または VSCode で実行できます。

## 実行手順

1. **ファイルのアップロード**: `ap_siken_all_items.csv` をアップロード
2. **モデルの選択**: 使用するモデルを設定
3. **セルを順番に実行**: すべてのセルを実行
4. **結果のダウンロード**: `ap_siken_all_items_vectors.csv` をダウンロード

## 1. 必要なライブラリのインストール

In [19]:
!pip install -q sentence-transformers pandas numpy tqdm

## 2. ライブラリのインポート

In [20]:
import pandas as pd
import numpy as np
import json
import time
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from google.colab import files
import os

## 3. 設定

In [21]:
# 使用するモデルを選択（コメントアウトを外して選択）
# MODEL_NAME = "pkshatech/GLuCoSE-base-ja"  # 日本語特化モデル
MODEL_NAME = "intfloat/multilingual-e5-large"  # 多言語対応モデル（推奨）
# MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # 軽量版

BATCH_SIZE = 32  # バッチサイズ（メモリに応じて調整）
TEXT_COLUMN = "問題名"  # ベクトル化する列名

## 4. CSVファイルのアップロード

**注意**: Google Colabで実行する場合、ファイルをアップロードしてください。
VSCodeで実行する場合は、このセルをスキップして次のセルで直接ファイルパスを指定してください。

In [22]:
# # Google Colab用: ファイルをアップロード
# uploaded = files.upload()
# input_filename = list(uploaded.keys())[0]
# print(f"アップロードされたファイル: {input_filename}")

In [23]:
# VSCode用: ローカルファイルを直接指定（上のセルの代わりにこちらを使用）
input_filename = "../01_scraping/ap_siken_all_items.csv"

## 5. データの読み込み

In [25]:
print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] データを読み込んでいます...")
df = pd.read_csv(input_filename, encoding='utf-8-sig')
print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] データ読み込み完了: {len(df)}行")
print(f"\nカラム: {list(df.columns)}")
print(f"\n最初の3行:")
display(df.head(3))

[2025-11-26 14:26:46] データを読み込んでいます...


FileNotFoundError: [Errno 2] No such file or directory: '../01_scraping/ap_siken_all_items.csv'

## 6. テキストの抽出

In [ ]:
if TEXT_COLUMN not in df.columns:
    raise ValueError(f"エラー: カラム '{TEXT_COLUMN}' が見つかりません。")

texts = df[TEXT_COLUMN].fillna('').tolist()
print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] テキスト抽出完了: {len(texts)}件")
print(f"\nサンプルテキスト:")
for i, text in enumerate(texts[:3]):
    print(f"{i+1}. {text}")

NameError: name 'df' is not defined

## 7. モデルのロード

In [ ]:
print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] モデル '{MODEL_NAME}' をロードしています...")
print("注意: 初回実行時はモデルのダウンロードが始まります（数分〜数十分かかる場合があります）")

model = SentenceTransformer(MODEL_NAME)

print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] モデルのロード完了")
print(f"ベクトル次元数: {model.get_sentence_embedding_dimension()}")

## 8. ベクトル化の実行

In [ ]:
print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] ベクトル化を開始します...")
print(f"総テキスト数: {len(texts)}")
print(f"バッチサイズ: {BATCH_SIZE}")

# ベクトル列名を生成
vector_column = f"vector_{MODEL_NAME.replace('/', '_').replace('.', '_').replace(':', '_')}"
print(f"ベクトル列名: {vector_column}")

# バッチ処理でベクトル化
all_vectors = []
num_batches = (len(texts) + BATCH_SIZE - 1) // BATCH_SIZE

start_time = time.time()

with tqdm(total=len(texts), desc="ベクトル化中") as pbar:
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i+BATCH_SIZE]
        batch_vectors = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False)
        all_vectors.extend(batch_vectors)
        pbar.update(len(batch_texts))

end_time = time.time()
duration = end_time - start_time

print(f"\n[{time.strftime('%Y-%m-%d %H:%M:%S')}] ベクトル化完了")
print(f"処理時間: {duration:.2f}秒")
print(f"1件あたり: {duration/len(texts):.3f}秒")

## 9. ベクトルをDataFrameに追加

In [ ]:
print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] ベクトルをDataFrameに追加しています...")

# ベクトルをJSON文字列に変換してDataFrameに追加
df[vector_column] = [json.dumps(vec.tolist()) for vec in all_vectors]

print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 追加完了")
print(f"\nDataFrameの形状: {df.shape}")
print(f"カラム数: {len(df.columns)}")

## 10. 結果の保存とダウンロード

In [ ]:
output_filename = "ap_siken_all_items_vectors.csv"

print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] CSVファイルを保存しています...")
df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 保存完了: {output_filename}")

# ファイルサイズを表示
file_size_mb = os.path.getsize(output_filename) / (1024 * 1024)
print(f"ファイルサイズ: {file_size_mb:.2f} MB")

# Google Colabの場合、ファイルをダウンロード
print("\nファイルをダウンロードしています...")
files.download(output_filename)
print("ダウンロード完了！")

## 11. 結果の確認（オプション）

In [ ]:
# ベクトルの次元数を確認
sample_vector = json.loads(df[vector_column].iloc[0])
print(f"ベクトルの次元数: {len(sample_vector)}")
print(f"\nサンプルベクトル（最初の10次元）:")
print(sample_vector[:10])

# データの統計情報
print(f"\n=== 処理結果サマリー ===")
print(f"総問題数: {len(df)}")
print(f"使用モデル: {MODEL_NAME}")
print(f"ベクトル次元数: {len(sample_vector)}")
print(f"処理時間: {duration:.2f}秒")
print(f"出力ファイル: {output_filename}")
print(f"ファイルサイズ: {file_size_mb:.2f} MB")

## 完了！

ベクトル化が完了しました。

### 次のステップ

1. ダウンロードした `ap_siken_all_items_vectors.csv` を `02_vectorize/output/` フォルダに配置
2. `03_html_output/main.py` を実行して類似度を計算
3. `index.html` をブラウザで開いて結果を確認